In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import Adam
import keras
from keras.optimizers.legacy import Adam as LegacyAdam

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pickle as pkl
from src.utils.data_transform import *
from src.utils.data_io import load_prediction_data
import pandas as pd
import os 
import pickle 
import json
from src.utils.data_io import save_data

In [2]:
results = []

In [3]:
with open("../data/dataset-info-json/subject_to_indices.json", "r") as f:
    subject_to_indices = json.load(f)

subject_to_indices = {int(k): v for k, v in subject_to_indices.items()}

In [4]:
path_to_data = "../data/lstm_training_data/processed/balanced_training_data.pkl"
path_to_models = "../models/lstm"

In [5]:
with open("../data/lstm_training_data/processed/balanced_training_data.pkl", "rb") as f:
    data = pickle.load(f)

In [6]:
for test_subject in subject_to_indices.keys():
    model_path = f"{path_to_models}/EFF_model_{test_subject}.keras"
    model = keras.models.load_model(model_path, compile=False)
    model.compile(optimizer=LegacyAdam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=["accuracy"])
    _, _, test_data, test_labels = load_prediction_data(test_subject, subject_to_indices, data)
    results.append(model.evaluate(test_data, test_labels))

2024-02-11 20:25:59.727228: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-02-11 20:25:59.727257: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-02-11 20:25:59.727270: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-02-11 20:25:59.727310: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-11 20:25:59.727328: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/opt/homebrew/Caskroom/miniforge/base/envs/tensorflowenv/lib/python3.10/site-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expec

8/8 [==============================] - 2s 31ms/step - loss: 0.0000e+00 - accuracy: 0.9876


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflowenv/lib/python3.10/site-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(32, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


2/2 [==============================] - 1s 27ms/step - loss: 0.0000e+00 - accuracy: 0.9688


In [7]:
print(f"Easy Accuracy: {100*np.mean([item[1] for item in results]):.2f}%")

Easy Accuracy: 95.80%


In [8]:
print(f"Loss: {100*np.mean([item[0] for item in results]):.2f}%")

Loss: 0.00%


Load training info

In [ ]:
tot_acc = []
tot_res = []
tot_loss = []
info_path = "../models/lstm/training_info/"
for i in range(1,13):
    with open(f"{info_path}/EFFaccuracy_{i}.pkl", "rb") as a:
        tot_acc.append(pickle.load(a))
    with open(f"{info_path}/EFFresults_{i}.pkl", "rb") as r:
        tot_res.append(pickle.load(r))
    with open(f"{info_path}/EFFloss_{i}.pkl", "rb") as l:
        tot_loss.append(pickle.load(l))

tot_res = np.concatenate(tot_res)
tot_acc = np.concatenate(tot_acc)
tot_loss = np.concatenate(tot_loss)

print(f"Evaluation: {np.mean(tot_res[:,1])*100}%")
print(f"Accuracy: {np.mean(tot_acc)*100}%")
print(f"Loss: {np.mean(tot_res[:,0])*100}%")